## Decorator
A decorator is a function that takes input another function. It may modify that function or return another function altogether


In [3]:
def deco(another_function):
    print("This is inside decorator")
    return another_function # always return the received function

@deco
def my_function():
    print("Inside my function")
    
my_function()

This is inside decorator
Inside my function


In most practical scenarios, decorators return a wrapped function like below. Important thing to note is decorators are executed at runtime even though the decorated functions are executed when they are called. 

In [23]:
def decorate(original_function):
    print("Inside decorator")
    def wrapper(*args, **kwargs):
        print("Inside wrapper")
        original_function(*args) # call the wrapped function 
    return wrapper

@decorate
def print_me(*send_anything):
    print(*send_anything)

print("Before calling print_me")
print_me("hi what's up", "123")

Inside decorator
Before calling print_me
Inside wrapper
hi what's up 123


## Decorators with args
You can also use decorators with arguments, like it is dont is Flask or Django. But you cannot simply pass in the argument list like @deco(arg1, arg2). To understand why it will fail, consider how it is translated

```
def decorate(original_function):
    print("Inside decorator")
    def wrapper(*args, **kwargs):
        print("Inside wrapper")
        original_function(*args) # call the wrapped function 
    return wrapper

@decorate
This is translated as:
print_me = decorate(print_me). So next time you're calling print_me you're actually calling wrapper(*args, **kwargs)
def print_me(*send_anything):
    print(*send_anything)

So if you do this:
@decorate(arg1, arg2)
That will be same as:
print_me = decorate(arg1, arg2)(print_me)
Which is undesired because it will end up in Nonetype not callable error. So you have to define decorator inside decorator
```

In [18]:
def decorate_with_args(*oargs):
    print("Inside decorator_with_args: ")
    print(*oargs)
    def wrap(original_function):
        print("Inside wrap")
        def wrapper(*args, **kwargs):
            print("Inside wrapper")
            original_function(*args) # call the wrapped function 
        return wrapper
    return wrap

@decorate_with_args("one", "two", "three")
# print_me = decorate_with_args("one", "two", "three")(print_me) above sentence is equal to this
def print_me(*send_anything):
    print(*send_anything)

print("Before calling print_me")
print_me("hi what's up", "123")

Inside decorator_with_args: 
one two three
Inside wrap
Before calling print_me
Inside wrapper
hi what's up 123


## Another inherent problem of decorators


In [27]:
def decorate(original_function):
    """This is a decorator"""
    def wrapper(*args, **kwargs):
        """This is a wrapper"""
        original_function(*args) # call the wrapped function 
    return wrapper

@decorate
def print_me(*send_anything):
    """This prints things"""
    print(*send_anything)

print(print_me.__name__)
print(print_me.__doc__)

wrapper
This is a wrapper


This is undesirable! Because the original metadata of the function is lost. To deal with this we use the functools wraps decorator! By passing the wraps function the original function we preserve its identity. 

In [31]:
from functools import wraps

def decorate(original_function):
    """This is a decorator"""
    @wraps(original_function)
    def wrapper(*args, **kwargs):
        """This is a wrapper"""
        original_function(*args) # call the wrapped function 
    return wrapper

@decorate
def print_me(*send_anything):
    """This prints things"""
    print(*send_anything)

print("Function name:", print_me.__name__)
print("Function doc:", print_me.__doc__)

Function name: print_me
Function doc: This prints things


You can also use decorator with class

In [29]:
class deco_class:
    def __init__(self, original_function):
        print("Inside init")
        print("Do stuff here..")
        self.original_function = original_function
        
    def __call__(self, *args, **kwargs):
        self.original_function(*args, **kwargs)
        
@deco_class
def print_me(message):
    print("Inside print me")
    print(message)

print_me("Hi this is Mayur")

Inside init
Do stuff here..
Inside print me
Hi this is Mayur


## Closure
Closure is basically tying the function to the environment. In other words, closure "closes" over the variables of outer function over inner functions so that the inner functions have access to the outer functions' variables even ___after___ the outer function is done execution. Decorators are nothing but closures. 

## Function scopes

In [2]:
# works perfectly fine
def my_func():
    b = 10
    def a():
        a = 5
        print(a)
        print(b) # Python understands this variable is from outer scope
    a()
my_func()

5
10


In [3]:
# doesn't work!
def my_func():
    b = 10
    def a():
        a = 5
        print(a)
        print(b)
        b = 2
    a()
my_func()

5


UnboundLocalError: local variable 'b' referenced before assignment

Above example fails even if the variable b is declared in the main function. This happens because Python treats the scope of variables in functions differently. In order to tell Python that it's the same variable that is defined in the outer scope, we need to use the nonlocal keyword. nonlocal call can be chained, for every nested function, nonlocal var will point to the variable outside that function. And the difference between nonlocal and global is that, global will make the var accessible anywhere throughout the program, nonlocal will work only in the nested functions. 

In [12]:
def my_func():
    b = 10
    def a():
        a = 5
        nonlocal b
        print(a)
        print("Inside a", b)
        b = 2
        print("Inside a. After assignment", b)
    a()
    print("Outside a", b)
my_func()

5
Inside a 10
Inside a. After assignment 2
Outside a 2


## Overloading in Python
By default, Python supports overriding, but not overloading. But since Python 3.4 there's been a new addition which enables to overload function 

In [15]:
from functools import singledispatch

@singledispatch
def print_me(arg):
    """This is the default fallback"""
    print("Default:", arg)

@print_me.register(int)
def int_print_me(arg):
    print("Int:", arg)
    
@print_me.register(str)
def str_print_me(arg):
    print("String:", arg)
    
@print_me.register(list)
def list_print_me(arg):
    print("List:", arg)
    
print_me("Hi!")
print_me(123)
print_me([1, 2, 3, 4])
print_me({'a': 1})

String: Hi!
Int: 123
List: [1, 2, 3, 4]
Default: {'a': 1}


## Stacked Decorators
Decorators can be stacked as well
```
@d1
@d2
def my_print():
    ...

is same as
my_print = d1(d2(my_print))
```